In [1]:
import pandas as pd
from collections import namedtuple

In [ ]:
def load_est_model_data():
    path = 'C:/Users/CNSPRHES/Documents/11-spreadsheets/EST model 2021 v5.xlsx'

    cols = [
        'Internregnskapskode', 'ACCOUNT', 'Cost center', 
        'Internregnskapskode beskrivelse', 'Region', 
        'Group Detail', 'Group', 'Account Group', 'HFMRapp1',
        'ACCOUNT_DESC', 'YearPeriodKey', 'Totalt'
    ]

    df = pd.read_excel(path, sheet_name='Data', usecols=cols)

    # Change column names
    new_cols = [str.casefold(c).replace(' ', '_') for c in cols]
    new_cols[-2] = 'year_period_key'
    df.columns = new_cols
    
    # Change dtype
    df.year_period_key = df.year_period_key.astype('str')
    
    return df

df = load_est_model_data()

In [ ]:
def create_query_string(filters):
    filtered = {k:v for k, v in filters.items() if v is not None}
    query = ' & '.join(f'{k}=="{v}"' for k, v in filtered.items() if k != 'year_period_key')
    return query + f' & year_period_key.isin({filtered["year_period_key"]})'
    
def filter_df(df, query_str):
    return (
        df
        .query(query_str)
        .groupby('year_period_key')[['totalt']]
        .sum()
        .transpose()
        .style
            .format("{:,.0f}")
    )
    
filters = dict(
    region='HJV', 
    internregnskapskode='R10T',
    group='Factory',
    group_detail=None,
    account_group='Revenue',
    year_period_key=[f'2021{str(i).zfill(2)}' for i in range(1, 13)]
)

    
query_str = create_query_string(filters)
filter_df(df, query_str)

### A more complete query string function:

In [ ]:
Constraint = namedtuple('Constraint', 'column, eq_sign, value')
  
def create_query_substring(c):
    if c.eq_sign in ['>', '<', '==']:
        return f'{c.column} {c.eq_sign} {repr(c.value)}'
    value = repr(c.value) if isinstance(c.value, str) else c.value
    return f'{c.column}.{c.eq_sign}({value})'

def create_query_string(constraints):
    return ' & '.join(create_query_substring(c) for c in constraints)
 

def filter_df(df, query_str):
    return (
        df
        .query(query_str)
        .groupby('year_period_key')[['totalt']]
        .sum()
        .transpose()
        .style
            .format("{:,.0f}")
    )

periods = [f'2021{str(i).zfill(2)}' for i in range(1, 13)]

constraints = [
    Constraint('region', '==', 'HJV'),
    Constraint('internregnskapskode', '==', 'R10T'),
    Constraint('group', 'str.contains', 'Factory'),
    Constraint('account_group', '==', 'Revenue'),
    Constraint('year_period_key', 'isin', periods)
]


query_str = create_query_string(constraints)
filter_df(df, query_str)

### Example 2:

In [ ]:
import pandas as pd
import operator
from functools import reduce

df = pd.DataFrame({"A": [6, 2, 10, -5, 3],
                   "B": [2, 5, 3, 2, 6],
                   "C": [-5, 2, 1, 8, 2]})

limits_dic = {"A": 0, "B": 2, "C": -1}

# equiv to [df['A'] > 0, df['B'] > 2 ...]
loc_elements = [df[key] > val for key, val in limits_dic.items()]

df = df.loc[reduce(operator.and_, loc_elements)]

### Example 3: 

In [ ]:
df = pd.DataFrame({"A": [6, 2, 10, -5, 3],
                   "B": [2, 5, 3, 2, 6],
                   "C": [-5, 2, 1, 8, 2]})

limits_dic = {"A": 0, "B": 2, "C": -1}

conditions = None
for key, val in limits_dic.items():
    cond = df[key] > val
    conditions = cond if conditions is None else conditions & cond

print(df[conditions].head())